# Set-up

In [1]:
# imports
import os
import sys
import synapseclient

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


# Login

In [2]:
# Token
SYNAPSE_AUTH_TOKEN = os.environ.get('SYNAPSE_AUTH_TOKEN')

In [3]:
# Login
syn = synapseclient.Synapse() 
syn.login(authToken=SYNAPSE_AUTH_TOKEN)


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.11.0) is available. Your version (2.7.2) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.11.0 release notes

https://python-docs.synapse.org/news/



Welcome, Adam Klie!



# Upload

In [ ]:
project_id = "syn64423137"

In [ ]:
datasets = [
    "Gersbach_WTC11-benchmark_TF-Perturb-seq_GEM-Xv3",
    "Gersbach_WTC11-benchmark_TF-Perturb-seq_HTv2",
    "Hon_WTC11-benchmark_TF-Perturb-seq",
    "Huangfu_WTC11-benchmark_TF-Perturb-seq",
]

base = "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets"


def get_children_map(parent_id):
    """Return {name: id} for direct children of a Synapse folder/project."""
    return {c["name"]: c["id"] for c in syn.getChildren(parent_id)}


def get_or_create_folder(parent_id, name):
    """Get a folder by name under parent_id, or create it if it doesn't exist."""
    children = get_children_map(parent_id)
    if name in children:
        return syn.get(children[name])
    return syn.store(synapseclient.Folder(name=name, parent=parent_id))


def mirror_to_synapse(local_root, syn_parent):
    """
    Recursively mirror local_root directory structure into Synapse folder syn_parent.
    local_root itself is *not* created as a folder; its contents are.
    """
    folder_map = {local_root: syn_parent}

    for dirpath, dirnames, filenames in os.walk(local_root):
        parent_id = folder_map[dirpath]

        # Create subfolders
        for d in dirnames:
            local_subdir = os.path.join(dirpath, d)
            syn_subfolder = syn.store(
                synapseclient.Folder(name=d, parent=parent_id)
            )
            folder_map[local_subdir] = syn_subfolder["id"]

        # Upload files
        for fname in filenames:
            local_file = os.path.join(dirpath, fname)
            syn_file = synapseclient.File(path=local_file, parent=parent_id)
            syn.store(syn_file)


# ---- Map existing top-level dataset folders under the project ----
top_level_children = get_children_map(project_id)

for dataset in datasets:
    dataset_dir = os.path.join(base, dataset)
    qc_dir = os.path.join(dataset_dir, "results", "2_qc")

    if not os.path.isdir(qc_dir):
        print(f"Skipping {dataset}: local qc directory missing at {qc_dir}")
        continue

    if dataset not in top_level_children:
        print(f"Skipping {dataset}: no matching folder under {project_id} on Synapse")
        continue

    dataset_syn_id = top_level_children[dataset]
    print(f"Uploading QC tree for dataset: {dataset} -> Synapse folder {dataset_syn_id}")

    # Ensure results/ and 2_qc/ folders exist under the dataset folder
    results_folder = get_or_create_folder(dataset_syn_id, "results")
    qc_folder = get_or_create_folder(results_folder["id"], "2_qc")

    # Mirror contents of local results/2_qc into Synapse results/2_qc
    mirror_to_synapse(qc_dir, qc_folder["id"])

    print(f"Finished uploading {dataset}")

Uploading QC tree for dataset: Gersbach_WTC11-benchmark_TF-Perturb-seq_GEM-Xv3 -> Synapse folder syn70110613

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################

In [28]:
dataset_name = "technology-benchmark_WTC11_TF-Perturb-seq"
local_results_dir = (
    "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/"
    "technology-benchmark_WTC11_TF-Perturb-seq/results"
)


def get_children_map(parent_id):
    """Return {name: id} for direct child folders of a Synapse container."""
    return {c["name"]: c["id"] for c in syn.getChildren(parent_id)}


def get_or_create_folder(parent_id, name):
    """Get a Synapse folder by name, or create it under parent_id."""
    children = get_children_map(parent_id)
    if name in children:
        return syn.get(children[name])
    return syn.store(synapseclient.Folder(name=name, parent=parent_id))


def mirror_to_synapse(local_root, syn_parent):
    """
    Recursively mirror the entire local directory tree starting at local_root
    *into* Synapse under syn_parent.
    """
    folder_map = {local_root: syn_parent}

    for dirpath, dirnames, filenames in os.walk(local_root):
        parent_id = folder_map[dirpath]

        # Create folders
        for d in dirnames:
            local_subdir = os.path.join(dirpath, d)
            syn_subfolder = syn.store(
                synapseclient.Folder(name=d, parent=parent_id)
            )
            folder_map[local_subdir] = syn_subfolder["id"]

        # Upload files
        for fname in filenames:
            file_path = os.path.join(dirpath, fname)
            syn_file = synapseclient.File(path=file_path, parent=parent_id)
            syn.store(syn_file)


# ============================ RUN UPLOAD ============================

# Ensure the dataset folder exists at the project top level
top_children = get_children_map(project_id)

if dataset_name not in top_children:
    raise ValueError(
        f"No folder named '{dataset_name}' under project {project_id}. "
        "Create it manually or correct the name."
    )

dataset_syn_id = top_children[dataset_name]

# Create or reuse dataset/results folder
results_folder = get_or_create_folder(dataset_syn_id, "results")

# Mirror the entire local results tree
print(f"Uploading entire results tree from {local_results_dir}")
mirror_to_synapse(local_results_dir, results_folder["id"])
print("Upload complete!")

Uploading entire results tree from /cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/technology-benchmark_WTC11_TF-Perturb-seq/results

##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


##################################################
 Uploading file to Synapse storage 
##################################################


################

In [5]:
import os
import hashlib
import synapseclient

# ---------------------------
# Helpers
# ---------------------------

def get_children_map(parent_id):
    """Return {name: id} for direct children of a Synapse folder/project."""
    return {c["name"]: c["id"] for c in syn.getChildren(parent_id)}

def get_or_create_folder_id(parent_id, name):
    """Get a folder by name under parent_id, or create it if it doesn't exist. Returns folder id."""
    children = get_children_map(parent_id)
    if name in children:
        return children[name]
    folder = syn.store(synapseclient.Folder(name=name, parent=parent_id))
    return folder["id"]

def md5_file(path, chunk_size=1024 * 1024):
    h = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(chunk_size), b""):
            h.update(chunk)
    return h.hexdigest()

def get_remote_file_md5(file_entity_id):
    """
    Return Synapse-stored MD5 for the current version of a File entity.
    Does not download the file.
    """
    ent = syn.get(file_entity_id, downloadFile=False)
    # Synapse File entity has 'md5' in properties for the file handle in most contexts
    # (If this ever returns None in your environment, we can fall back to fileHandles API.)
    return ent.properties.get("md5")

# ---------------------------
# Incremental mirroring
# ---------------------------

def mirror_to_synapse_incremental(local_root, syn_parent_id, *, compare_md5=False, verbose=True):
    """
    Recursively mirror local_root directory structure into Synapse folder syn_parent_id.
    Only uploads files that do NOT already exist by name in the destination folder.
    If compare_md5=True, uploads only when the existing file's MD5 differs.
    """
    # Map local dir -> syn folder id
    folder_map = {os.path.abspath(local_root): syn_parent_id}

    for dirpath, dirnames, filenames in os.walk(local_root):
        dirpath = os.path.abspath(dirpath)
        parent_syn_id = folder_map[dirpath]

        # Create/reuse subfolders (no duplicates)
        # Note: keep os.walk from descending into dirs you might want to skip by editing dirnames in-place.
        for d in dirnames:
            local_subdir = os.path.join(dirpath, d)
            syn_sub_id = get_or_create_folder_id(parent_syn_id, d)
            folder_map[os.path.abspath(local_subdir)] = syn_sub_id

        # Build one-time map of existing children under this Synapse folder
        syn_children = get_children_map(parent_syn_id)  # name -> id

        # Upload only missing / changed files
        for fname in filenames:
            local_file = os.path.join(dirpath, fname)
            if not os.path.isfile(local_file):
                continue

            if fname in syn_children:
                # Something with same name exists (could be File or Folder; skip either way)
                existing_id = syn_children[fname]

                if compare_md5:
                    try:
                        remote_md5 = get_remote_file_md5(existing_id)
                    except Exception:
                        remote_md5 = None

                    local_md5 = md5_file(local_file)

                    if remote_md5 and remote_md5.lower() == local_md5.lower():
                        if verbose:
                            print(f"SKIP (same md5): {local_file}")
                        continue
                    else:
                        # If same name exists but different content, you need a policy:
                        # (a) skip, (b) upload as a NEW entity w/ different name, or (c) update version.
                        # Here we update the existing File entity by storing with its id.
                        if verbose:
                            print(f"UPDATE (md5 differs): {local_file} -> {existing_id}")
                        f_ent = synapseclient.File(path=local_file, parent=parent_syn_id)
                        f_ent.id = existing_id
                        syn.store(f_ent, forceVersion=True)  # creates a new version
                        continue

                if verbose:
                    print(f"SKIP (exists): {local_file}")
                continue

            # New file by name -> upload
            if verbose:
                print(f"UPLOAD (new): {local_file}")
            syn_file = synapseclient.File(path=local_file, parent=parent_syn_id)
            syn.store(syn_file)

In [6]:
# ---------------------------
# Your loop (minimal changes)
# ---------------------------

datasets = [
    #"Gersbach_WTC11-benchmark_TF-Perturb-seq_GEM-Xv3",
    #"Gersbach_WTC11-benchmark_TF-Perturb-seq_HTv2",
    "Hon_WTC11-benchmark_TF-Perturb-seq",
    "Huangfu_WTC11-benchmark_TF-Perturb-seq",
]

base = "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets"

top_level_children = get_children_map(project_id)

for dataset in datasets:
    dataset_dir = os.path.join(base, dataset)
    qc_dir = os.path.join(dataset_dir, "results", "2_qc")

    if not os.path.isdir(qc_dir):
        print(f"Skipping {dataset}: local qc directory missing at {qc_dir}")
        continue

    if dataset not in top_level_children:
        print(f"Skipping {dataset}: no matching folder under {project_id} on Synapse")
        continue

    dataset_syn_id = top_level_children[dataset]
    print(f"Uploading QC tree for dataset: {dataset} -> Synapse folder {dataset_syn_id}")

    # Ensure results/ and 2_qc/ folders exist under the dataset folder
    results_id = get_or_create_folder_id(dataset_syn_id, "results")
    qc_id = get_or_create_folder_id(results_id, "2_qc")

    # Incremental mirror
    mirror_to_synapse_incremental(qc_dir, qc_id, compare_md5=False, verbose=True)
    # If you want “only upload when content differs”, use:
    # mirror_to_synapse_incremental(qc_dir, qc_id, compare_md5=True, verbose=True)

    print(f"Finished uploading {dataset}")

Uploading QC tree for dataset: Hon_WTC11-benchmark_TF-Perturb-seq -> Synapse folder syn70110603
SKIP (exists): /cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/results/2_qc/2025_12_01/mapping_qc/pre_filter_guide_qc.png
SKIP (exists): /cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/results/2_qc/2025_12_01/mapping_qc/pre_filter_gene_knee_plot.png
SKIP (exists): /cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/results/2_qc/2025_12_01/mapping_qc/gene.filtered.h5ad
SKIP (exists): /cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/results/2_qc/2025_12_01/mapping_qc/post_filter_gene_qc_histograms_by_batch.png
SKIP (exists): /cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/results/2_qc/2025_12_01/mapping_qc/guide.filtered.h5ad
SKIP (exists): /cellar/users/aklie/data/datasets/tf_